In [ ]:
#run to import all the stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,gc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve,auc,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import time
plt.style.use('seaborn-whitegrid')
import warnings
warnings.simplefilter("ignore")
colors = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

In [ ]:
#run this command each time to mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#run this command each time to set the directory
import os
os.chdir('/content/drive/MyDrive/kaggle/fraud-detection')

In [ ]:
df_train_transaction = pd.read_csv('data/train_transaction.csv')
df_train_identity = pd.read_csv('data/train_identity.csv')
print(f'Shape of transaction train data: {df_train_transaction.shape}')
print(f'Shape of identity train data: {df_train_identity.shape}')

Shape of transaction train data: (590540, 394)
Shape of identity train data: (144233, 41)


In [ ]:
df_train = df_train_transaction.merge(df_train_identity,on=['TransactionID'],how='left')
df_train.to_csv('data/train_combined.csv',index=False)
df_train.head()
gc.collect()

11

In [ ]:
df_test_transaction = pd.read_csv('data/test_transaction.csv')
df_test_identity = pd.read_csv('data/test_identity.csv')

In [ ]:
df_test = df_test_transaction.merge(df_test_identity,on=['TransactionID'],how='left')
df_test.to_csv('data/test_combined.csv',index=False)
gc.collect()
df_test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,NaN,231.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,472.0,87.0,2635.0,NaN,hotmail.com,NaN,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,NaN,NaN,NaN,NaN,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,205.0,87.0,17.0,NaN,gmail.com,NaN,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,NaN,NaN,NaN,NaN,242.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,264.0,87.0,6.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,NaN,NaN,NaN,NaN,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
req_cols = [col for col in df_train.columns if df_train[col].isnull().sum()/df_train.shape[0] < 0.7]
df_train = df_train[req_cols]
req_cols.remove('isFraud')
df_test = df_test[req_cols]
print(df_train.shape)
print(df_test.shape)

(590540, 226)
(506691, 225)


In [ ]:
df_train.to_csv('data/train_combined.csv',index=False)
df_test.to_csv('data/test_combined.csv',index=False)

In [ ]:
df_train = pd.read_csv('data/train_combined.csv')
df_test = pd.read_csv('data/test_combined.csv')

In [ ]:
X_train = df_train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y_train = df_train.sort_values('TransactionDT')['isFraud']
X_test = df_test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)

print(X_train.shape)
print(X_test.shape)
gc.collect()

(590540, 223)
(506691, 223)


99

In [ ]:
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X_train = clean_inf_nan(X_train)
X_test = clean_inf_nan(X_test)

print(X_train.shape)
print(X_test.shape)

(590540, 223)
(506691, 223)


In [ ]:
X_train.to_csv('data/X_train.csv',index=False)
y_train.to_csv('data/y_train.csv',index=False)
X_test.to_csv('data/X_test.csv',index=False)

In [ ]:
X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')
y_train = pd.read_csv('data/y_train.csv')

In [ ]:
del df_train,df_test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_,X_cv_,y_train_,y_cv_ = train_test_split(X_train,y_train, stratify = y_train,test_size=0.3, random_state=40)

print(X_train_.shape)
print(X_cv_.shape)
print(y_train_.shape)
print(y_cv_.shape)

(413378, 223)
(177162, 223)
(413378,)
(177162,)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_cols = X_train.select_dtypes(exclude=object).columns.tolist()


# 1. Median imputation for numarical features
# 2. Standardizing


# Featurization of numeric data
imputer = SimpleImputer(strategy='median')
x_train_n = imputer.fit_transform(X_train_[num_cols])
x_cv_n = imputer.transform(X_cv_[num_cols])
x_test_n = imputer.transform(X_test[num_cols])

scaler = StandardScaler()
x_train_nums = scaler.fit_transform(x_train_n)
x_cv_nums = scaler.transform(x_cv_n)
x_test_nums = scaler.transform(x_test_n)


x_train_num_final = pd.DataFrame(x_train_nums, columns=num_cols)
x_cv_num_final = pd.DataFrame(x_cv_nums, columns=num_cols)
x_test_num_final = pd.DataFrame(x_test_nums, columns=num_cols)

gc.collect()

print(x_train_num_final.shape)
print(x_cv_num_final.shape)
print(x_test_num_final.shape)

(413378, 210)
(177162, 210)
(506691, 210)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# we are manually giving datatype because some categorical columns are labelled as float/int
cat_cols = ['ProductCD','card1','card2','card3','card4','card5','card6','P_emaildomain','R_emaildomain','M1',
'M2','M3','M4','M5','M6','M7','M8','M9','id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29','addr1','addr2',
'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

for col in cat_cols:
    if col in X_train.columns:
        X_train_[col] = X_train_[col].astype('object')
        X_cv_[col] = X_cv_[col].astype('object')
        X_test[col] = X_test[col].astype('object')
        
        
        X_train_[col]= X_train_[col].fillna('MISSING')
        X_cv_[col]= X_cv_[col].fillna('MISSING')
        X_test[col] = X_test[col].fillna('MISSING')
        
        lb = LabelEncoder()
        lb.fit(list(X_train_[col].values) + list(X_test[col].values) + list(X_cv_[col].values))
        X_train_[col] = lb.transform(list(X_train_[col].values))
        X_cv_[col] = lb.transform(list(X_cv_[col].values)) 
        X_test[col] = lb.transform(list(X_test[col].values))

In [ ]:
def returnCatNumList(df):
    
    object_cols = df.select_dtypes(include="object").columns.tolist()
    numeric_cols = df.select_dtypes(exclude="object").columns.tolist()

    return object_cols, numeric_cols

In [ ]:
# Final complete data
x_train_num_final = x_train_num_final.reset_index(drop=True)
x_cv_num_final = x_cv_num_final.reset_index(drop=True)
x_test_num_final = x_test_num_final.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)
X_cv_ = X_cv_.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

object_cols, numeric_cols = returnCatNumList(X_train_)
x_train = pd.concat([x_train_num_final,X_train_[object_cols]], axis = 1)
object_cols, numeric_cols = returnCatNumList(X_cv_)
x_cv = pd.concat([x_cv_num_final,X_cv_[object_cols]], axis = 1)
object_cols, numeric_cols = returnCatNumList(X_test)
x_test = pd.concat([x_test_num_final,X_test[object_cols]], axis = 1)

print(x_train.shape)
print(x_cv.shape)
print(x_test.shape)

(537388, 210)
(177162, 210)
(506691, 210)


In [ ]:
x_train.replace([np.inf, -np.inf], np.nan, inplace=True)
x_train.dropna(inplace=True)
y_train.replace([np.inf, -np.inf], np.nan, inplace=True)
y_train.dropna(inplace=True)

best_est = LogisticRegression(solver='liblinear',class_weight='balanced')
best_est = best_est.fit(x_train,y_train)
y_pred = best_est.predict(x_cv)

train_fpr, train_tpr, thresholds = roc_curve(y_train, best_est.predict_proba(x_train)[:,1])
test_fpr, test_tpr, thresholds = roc_curve(y_cv, best_est.predict_proba(x_cv)[:,1])
plt.grid(True)
plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.title("ROC CURVE FOR OPTIMAL K")
plt.show()

#Area under ROC curve
print('Area under train roc {}'.format(auc(train_fpr, train_tpr)))
print('Area under test roc {}'.format(auc(test_fpr, test_tpr)))

ValueError: ignored